In [1]:
#because CRAFT can only detects text, to make a fair comparision 
#between CRAFT-based method and Faster-R-CNN-based method, we need to 
#concatenate non-text resutls from Faster-R-CNN-based method and 
#text results from CRAFT-based method 

import csv
class Point:
    def __init__(self,setX,setY):
        self.x = setX
        self.y = setY
    def printPoint(self):
        print("x: "+str(self.x))
        print("y: "+str(self.y))
class BoundingBox:
    def __init__(self,setLogicalClass,setXLeft,setXRight,setYTop,setYBottom):
        self.logicalClass = setLogicalClass
        self.XLeft = setXLeft
        self.XRight = setXRight
        self.YTop = setYTop
        self.YBottom = setYBottom
    def printBox(self):
        print("logicalClass: "+str(self.logicalClass))
        print("x_left: "+str(self.XLeft))
        print("x_right: "+str(self.XRight))
        print("y_top: "+str(self.YTop))
        print("y_bottom: "+str(self.YBottom))



In [2]:
#collect bounding boxes information from both CRAFT and Faster R-CNN
def collect_boxes_and_labels(text_only_csv_file_name,mixed_csv_file_name):
    Bounding_box_list = []
    labels_list = []
    with open(mixed_csv_file_name, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if(row['label']=="2"):
#                 print("2")
                x_left = row['xmin']
                x_right = row['xmax']
                y_top = row['ymin']
                y_bottom = row['ymax']
                bbx = BoundingBox(2,x_left,x_right,y_top,y_bottom)
                Bounding_box_list.append(bbx)
                labels_list.append(2)
    with open(text_only_csv_file_name, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            x_left = row['xmin']
            x_right = row['xmax']
            y_top = row['ymin']
            y_bottom = row['ymax']
            bbx = BoundingBox(1,x_left,x_right,y_top,y_bottom)
            Bounding_box_list.append(bbx)
            labels_list.append(1)     
    return Bounding_box_list, labels_list

In [3]:
#output concatenated results
Bounding_boxes_lists_all = []
labels_lists_all = []
number_of_images = 300
for i in range(number_of_images):
    text_only_csv_file_name = "./bce_csv_text_only/"+str(i)+"_boxes.csv"
    mixed_csv_file_name = "./BCE_test/result_boxes/"+str(i)+"_boxes.csv"
    Bounding_box_list = []
    labels_list = []
    Bounding_box_list,labels_list = collect_boxes_and_labels(text_only_csv_file_name,mixed_csv_file_name)
    Bounding_boxes_lists_all.append(Bounding_box_list)
    labels_lists_all.append(labels_list)

In [4]:
#in .csv format
import csv
def write_box_info_into_csv(boxes_accepted_list,labels_list,saving_root_boxes):
    for i in range(len(boxes_accepted_list)):
        with open(saving_root_boxes+str(i)+'_boxes.csv', 'w', newline='') as csvfile:
            fieldnames = ['xmin', 'ymin','xmax','ymax','label']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            for j in range(len(boxes_accepted_list[i])):
                xmin = boxes_accepted_list[i][j].XLeft
                ymin = boxes_accepted_list[i][j].YTop
                xmax = boxes_accepted_list[i][j].XRight
                ymax = boxes_accepted_list[i][j].YBottom
                writer.writerow({'xmin': str(int(xmin)), 'ymin': str(int(ymin)),'xmax': str(int(xmax)),'ymax': str(int(ymax)),'label':str(int(labels_list[i][j]))})

write_box_info_into_csv(Bounding_boxes_lists_all,labels_lists_all,"./cnnSeg/bce_csv_mixed/")